In [51]:
import excelHelper
import importlib, pandas, sys, re, os
importlib.reload(excelHelper)
helper = excelHelper.excelHelper()

def get_df(fname, sheetname=None, index=False):
    
    if sheetname is None:
        df = pandas.read_excel(fname, index=index)
    else:
        df = pandas.read_excel(fname, sheetname=sheetname, index=index)
        
    print(df.head())
    
    if 'baseMean' in df.columns:
        df['Mean frequency'] = df['baseMean']
        del df['baseMean']
    
    for col in ['lfcSE', 'pvalue', 'stat']:
        del df[col]

    df['Gene name'] = df.index
    if 'padj' in df.columns:
        df['p value (adjusted)'] = df['padj']
        del df['padj']
    
    df = df[['Gene name', 'Mean frequency', 'log2FoldChange', 'p value (adjusted)']]
    return df
in_file = '/Users/dfporter/Desktop/macbook_air_Desktop/shared/sp_oo/FBF_gendered_gl/tables/File S4 raw DESeq2 output.xlsx'
df = get_df(in_file, sheetname=None, index=True)


helper.add_gene_description(df)
del df['transcript_id']

print('\n>>>\n', df)
oo_up = df[df['log2FoldChange']<=-1]
oo_up = oo_up[oo_up['p value (adjusted)']<=0.01]

sp_up = df[df['log2FoldChange']>=1]
sp_up = sp_up[sp_up['p value (adjusted)']<=0.05]


            baseMean  log2FoldChange     lfcSE       stat        pvalue  \
clec-87   477.187118       -4.007144  0.211601 -18.937294  5.621309e-80   
snpc-1.3  415.796892        4.524025  0.262932  17.206052  2.391951e-66   
mrp-4     338.825486       -5.008864  0.302134 -16.578275  1.000607e-61   
meg-1     313.353018       -5.908888  0.409290 -14.436936  3.030150e-47   
fog-1     923.307044        3.030068  0.211325  14.338428  1.258741e-46   

                  padj  
clec-87   5.689327e-76  
snpc-1.3  1.210447e-62  
mrp-4     3.375715e-58  
meg-1     7.667038e-44  
fog-1     2.547943e-43  

>>>
            Gene name  Mean frequency  log2FoldChange  p value (adjusted)  \
clec-87      clec-87      477.187118       -4.007144        5.689327e-76   
snpc-1.3    snpc-1.3      415.796892        4.524025        1.210447e-62   
mrp-4          mrp-4      338.825486       -5.008864        3.375715e-58   
meg-1          meg-1      313.353018       -5.908888        7.667038e-44   
fog-1        

In [52]:
        
writer = pandas.ExcelWriter(
    '/Users/dfporter/Desktop/macbook_air_Desktop/shared/sp_oo/FBF_gendered_gl/tables/File S4 DESeq2 SP vs OO.xlsx')


sp_up.to_excel(
    writer,
    sheet_name='SP enriched',
    index=False,
    #columns=tab3_columns
)

oo_up.to_excel(
    writer,
    sheet_name='OO enriched',
    index=False,
    #columns=tab3_columns
)

df.to_excel(
    writer,
    sheet_name='All DESeq2 results',
    index=False,

)
print(oo_up)

writer.save()

             Gene name  Mean frequency  log2FoldChange  p value (adjusted)  \
clec-87        clec-87      477.187118       -4.007144        5.689327e-76   
mrp-4            mrp-4      338.825486       -5.008864        3.375715e-58   
meg-1            meg-1      313.353018       -5.908888        7.667038e-44   
ZK1010.t1    ZK1010.t1      296.268379       -4.635183        4.129361e-36   
cpg-2            cpg-2      714.042490       -2.325217        1.150347e-34   
puf-5            puf-5     1125.227613       -2.304972        6.051913e-30   
pos-1            pos-1      310.546115       -2.368572        2.355870e-27   
F14H3.6        F14H3.6       70.918500       -5.426825        2.627319e-26   
W05F2.3        W05F2.3      225.569157       -2.252567        2.417593e-25   
die-1            die-1      105.013750       -3.510844        4.800231e-25   
cpg-3            cpg-3       95.356401       -3.457000        4.962231e-23   
C01G8.1        C01G8.1      447.998958       -2.035978        8.

In [53]:



in_file = '/Users/dfporter/Desktop/macbook_air_Desktop/shared/sp_oo/FBF_gendered_gl/tables/Table S6 Blocks.xls'

xls = pandas.ExcelFile(in_file)
print(xls.sheet_names)

df_by_sheetname = {}
for sheetname in xls.sheet_names:
    df = pandas.read_excel(in_file, sheetname=sheetname, index=False)
    helper.add_gene_description(df)
    del df['transcript_id']
    print(df)
    df_by_sheetname[sheetname] = df.copy()
    
    
writer = pandas.ExcelWriter(
    '/Users/dfporter/Desktop/macbook_air_Desktop/shared/sp_oo/FBF_gendered_gl/tables/File S6 Blocks.xlsx')

for sheetname, df in df_by_sheetname.items():
    df.to_excel(
        writer,
        sheet_name=sheetname,
        index=False,
        #columns=tab3_columns
    )
    
writer.save()
#print(df.head(2))

['Block I', 'Block II', 'Block III']
      Gene name                                        Description
0       enu-3.5                        ENhancer of Uncoordination 
1         rpl-2                          60S ribosomal protein L8 
2        col-94                                          COLlagen 
3       F22B8.7                                                   
4         fog-3           FOG-3 protein; Feminization Of Germline 
5       R11F4.1                          Probable glycerol kinase 
6      Y47D3A.5                                                   
7      R04B5.11                                                   
8       Y4C6A.4                                                   
9      C09H10.9                                                   
10      F32B5.4                                MS Related Protein 
11      C44B9.2                                                   
12     F52F12.5                                                   
13     F47D12.7  Kelch re

   Gene name                                        Description
0      gld-1   Female germline-specific tumor suppressor gld-1 
1     lin-41                                                   
2      cpg-1                        Chondroitin proteoglycan 1 
3      plk-3             Serine/threonine-protein kinase plk-3 
4      cgh-1                  ATP-dependent RNA helicase cgh-1 
5     wago-4                                 Piwi-like protein 
6     ifet-1                    Translational repressor ifet-1 
7      cpb-3  Cytoplasmic polyadenylation element-binding pr...
8      ppw-2                                 Piwi-like protein 
9      ima-3                          Importin subunit alpha-3 
10     car-1            Cytokinesis, Apoptosis, RNA-associated 
11    trcs-1             TRansport of membrane to Cell Surface 
12     egg-6      Leucine-rich repeat-containing protein egg-6 
13    spat-2                      Suppressor of PAr-Two defect 
14     ncl-1              B-box type zin

In [60]:

in_file = '/Users/dfporter/Desktop/macbook_air_Desktop/shared/sp_oo/FBF_gendered_gl/orthos/output/File S7 conservation.xls'

xls = pandas.ExcelFile(in_file)
print(xls.sheet_names)


df_by_sheetname = {}
for sheetname in xls.sheet_names:
    df = pandas.read_excel(in_file, sheetname=sheetname, index=False)
    df['Gene name'] = df['Worm gene name']
    
    helper.add_gene_description(df)
    
    df['Worm gene description'] = df['Description']
    
    for col in ['transcript_id', 'Description', 'Gene name']:
        del df[col]
        
    print(df)
    df_by_sheetname[sheetname] = df.copy()
    
    
writer = pandas.ExcelWriter(
    '/Users/dfporter/Desktop/macbook_air_Desktop/shared/sp_oo/FBF_gendered_gl/tables/xFile S6 Blocks.xlsx')

for sheetname, df in df_by_sheetname.items():
    df.to_excel(
        writer,
        sheet_name=sheetname,
        index=False,
        #columns=tab3_columns
    )
    
writer.save()
#print(df.head(2))

['OO FBF (25°C)', 'OO FBF (25°C) top 500', 'SP FBF (25°C)', 'SP FBF (25°C) top 500', 'OO FBF-1 (20°C)', 'OO FBF-1 (20°C) top 500', 'OO FBF-2 (20°C)', 'OO FBF-2 (20°C) top 500', 'Block I', 'Block II', 'Block III', 'Common', 'Common top 500']
    Worm locus ID Worm gene name       Human ortholog Ensembl IDs  \
0         B0041.5        B0041.5                  ENSG00000115084   
1         B0464.6        B0464.6                  ENSG00000146842   
2         B0491.1        B0491.1                  ENSG00000143315   
3         B0511.6        B0511.6                  ENSG00000088205   
4         C04G6.4        C04G6.4                  ENSG00000164244   
5        C16C10.4       C16C10.4                  ENSG00000150459   
6        C33A12.3       C33A12.3                  ENSG00000151470   
7        C34B2.10       C34B2.10                  ENSG00000114902   
8         C35A5.8        C35A5.8  ENSG00000130227,ENSG00000204764   
9         C44E4.4        C44E4.4                  ENSG00000138385   


   Worm locus ID Worm gene name       Human ortholog Ensembl IDs  \
0       C33A12.3       C33A12.3                  ENSG00000151470   
1       C34B2.10       C34B2.10                  ENSG00000114902   
2        C35A5.8        C35A5.8  ENSG00000130227,ENSG00000204764   
3        C44E4.4        C44E4.4                  ENSG00000138385   
4       F11A10.5       F11A10.5  ENSG00000004866,ENSG00000007341   
5      F17C11.10      F17C11.10                  ENSG00000198554   
6        F25B4.5        F25B4.5                  ENSG00000185246   
7       F37B12.3       F37B12.3                  ENSG00000153989   
8        F52A8.5        F52A8.5                  ENSG00000140905   
9       K07H8.10       K07H8.10  ENSG00000165502,ENSG00000115053   
10       R05D3.2        R05D3.2  ENSG00000139636,ENSG00000105983   
11        R166.3         R166.3  ENSG00000144233,ENSG00000101935   
12       T01G9.2        T01G9.2                  ENSG00000125149   
13       T04C4.1        T04C4.1  ENSG00000217128

    Worm locus ID Worm gene name       Human ortholog Ensembl IDs  \
0         B0491.1        B0491.1                  ENSG00000143315   
1         C04G6.4        C04G6.4                  ENSG00000164244   
2        C33A12.3       C33A12.3                  ENSG00000151470   
3        C34B2.10       C34B2.10                  ENSG00000114902   
4         C35A5.8        C35A5.8  ENSG00000130227,ENSG00000204764   
5         C44E4.4        C44E4.4                  ENSG00000138385   
6         C53B4.4        C53B4.4                  ENSG00000165650   
7         D1007.5        D1007.5  ENSG00000121775,ENSG00000176142   
8         D1054.3        D1054.3                  ENSG00000165416   
9         D1069.3        D1069.3  ENSG00000155744,ENSG00000122591   
10        D2085.4        D2085.4                  ENSG00000009335   
11        E04D5.1        E04D5.1                  ENSG00000144895   
12        F07F6.4        F07F6.4  ENSG00000149182,ENSG00000242247   
13       F11A10.5       F11A10.5  

   Worm locus ID Worm gene name       Human ortholog Ensembl IDs  \
0        B0491.1        B0491.1                  ENSG00000143315   
1       C34B2.10       C34B2.10                  ENSG00000114902   
2        C35A5.8        C35A5.8  ENSG00000130227,ENSG00000204764   
3        C44E4.4        C44E4.4                  ENSG00000138385   
4        E04D5.1        E04D5.1                  ENSG00000144895   
5      F17C11.10      F17C11.10                  ENSG00000198554   
6        F25B4.5        F25B4.5                  ENSG00000185246   
7       F37B12.3       F37B12.3                  ENSG00000153989   
8        F52A8.5        F52A8.5                  ENSG00000140905   
9       K07H8.10       K07H8.10  ENSG00000165502,ENSG00000115053   
10       R05D3.2        R05D3.2  ENSG00000139636,ENSG00000105983   
11        R166.3         R166.3  ENSG00000144233,ENSG00000101935   
12     Y48G1C.12      Y48G1C.12                  ENSG00000164162   
13      Y71H2B.2       Y71H2B.2                 

    Worm locus ID Worm gene name       Human ortholog Ensembl IDs  \
0         B0041.5        B0041.5                  ENSG00000115084   
1         B0491.1        B0491.1                  ENSG00000143315   
2         C04G6.4        C04G6.4                  ENSG00000164244   
3         C05C8.1        C05C8.1                  ENSG00000125375   
4         C05C8.2        C05C8.2                  ENSG00000111615   
5        C05D2.10       C05D2.10                  ENSG00000047056   
6        C16C10.4       C16C10.4                  ENSG00000150459   
7        C16C10.8       C16C10.8                  ENSG00000145220   
8         C18E3.5        C18E3.5  ENSG00000227709,ENSG00000060688   
9         C24G6.8        C24G6.8                  ENSG00000141378   
10       C33A12.3       C33A12.3                  ENSG00000151470   
11       C34B2.10       C34B2.10                  ENSG00000114902   
12      C34E10.10      C34E10.10                  ENSG00000147679   
13        C35A5.8        C35A5.8  

...

    Worm locus ID Worm gene name       Human ortholog Ensembl IDs  \
0         B0041.5        B0041.5                  ENSG00000115084   
1         B0491.1        B0491.1                  ENSG00000143315   
2         B0511.6        B0511.6                  ENSG00000088205   
3         C04G6.4        C04G6.4                  ENSG00000164244   
4         C05C8.2        C05C8.2                  ENSG00000111615   
5        C05D2.10       C05D2.10                  ENSG00000047056   
6         C11D2.4        C11D2.4                  ENSG00000165118   
7         C16A3.6        C16A3.6                  ENSG00000198042   
8        C16C10.4       C16C10.4                  ENSG00000150459   
9         C18E3.5        C18E3.5  ENSG00000227709,ENSG00000060688   
10        C24G6.8        C24G6.8                  ENSG00000141378   
11        C30B5.2        C30B5.2  ENSG00000104660,ENSG00000213625   
12       C33A12.3       C33A12.3                  ENSG00000151470   
13      C34E10.10      C34E10.10  

   Worm locus ID Worm gene name       Human ortholog Ensembl IDs  \
0        C44E4.4        C44E4.4                  ENSG00000138385   
1      F17C11.10      F17C11.10                  ENSG00000198554   
2       F37B12.3       F37B12.3                  ENSG00000153989   
3        F52A8.5        F52A8.5                  ENSG00000140905   
4       K03H1.13       K03H1.13                  ENSG00000254685   
5       K04F10.7       K04F10.7  ENSG00000009780,ENSG00000077458   
6       K07H8.10       K07H8.10  ENSG00000165502,ENSG00000115053   
7         R166.3         R166.3  ENSG00000144233,ENSG00000101935   
8        T04C4.1        T04C4.1  ENSG00000217128,ENSG00000052795   
9      Y48G1C.12      Y48G1C.12                  ENSG00000164162   
10    Y62E10A.13     Y62E10A.13                  ENSG00000146733   
11      Y71H2B.2       Y71H2B.2                  ENSG00000164902   
12      F41E6.13         atg-18  ENSG00000070540,ENSG00000157954   
13       R06F6.1          cdl-1                 

  Worm locus ID Worm gene name       Human ortholog Ensembl IDs  \
0       F21F3.3        F21F3.3                  ENSG00000116237   
1       F09G2.3         pitr-5  ENSG00000144136,ENSG00000168575   
2      B0361.10          ykt-6                  ENSG00000106636   

  PUM2 target HGNC symbol                             Worm gene description  
0                    ICMT  Protein-S-isoprenylcysteine O-methyltransferase   
1        SLC20A1, SLC20A2                            Phosphate transporter   
2                    YKT6                     YKT6 (Yeast v-SNARE) homolog   
    Worm locus ID Worm gene name       Human ortholog Ensembl IDs  \
0         B0491.1        B0491.1                  ENSG00000143315   
1         C35A5.8        C35A5.8  ENSG00000130227,ENSG00000204764   
2         C44E4.4        C44E4.4                  ENSG00000138385   
3         C53B4.4        C53B4.4                  ENSG00000165650   
4         E04D5.1        E04D5.1                  ENSG00000144895   
5    

  Worm locus ID Worm gene name       Human ortholog Ensembl IDs  \
0       C07H6.5          cgh-1  ENSG00000110367,ENSG00000269612   
1       F56F3.1         ifet-1                  ENSG00000184708   
2        R144.7         larp-1  ENSG00000155506,ENSG00000138709   
3       ZK112.2          ncl-1  ENSG00000109654,ENSG00000110171   

  PUM2 target HGNC symbol                  Worm gene description  
0                    DDX6      ATP-dependent RNA helicase cgh-1   
1               EIF4ENIF1        Translational repressor ifet-1   
2                   LARP1                  La-related protein 1   
3                   TRIM2  B-box type zinc finger protein ncl-1   
    Worm locus ID Worm gene name       Human ortholog Ensembl IDs  \
0         B0491.1        B0491.1                  ENSG00000143315   
1         C04G6.4        C04G6.4                  ENSG00000164244   
2        C33A12.3       C33A12.3                  ENSG00000151470   
3        C34B2.10       C34B2.10                  ENS

   Worm locus ID Worm gene name       Human ortholog Ensembl IDs  \
0       C33A12.3       C33A12.3                  ENSG00000151470   
1       C34B2.10       C34B2.10                  ENSG00000114902   
2        C35A5.8        C35A5.8  ENSG00000130227,ENSG00000204764   
3        C44E4.4        C44E4.4                  ENSG00000138385   
4       F11A10.5       F11A10.5  ENSG00000004866,ENSG00000007341   
5      F17C11.10      F17C11.10                  ENSG00000198554   
6        F25B4.5        F25B4.5                  ENSG00000185246   
7       F37B12.3       F37B12.3                  ENSG00000153989   
8        F52A8.5        F52A8.5                  ENSG00000140905   
9       K07H8.10       K07H8.10  ENSG00000165502,ENSG00000115053   
10       R05D3.2        R05D3.2  ENSG00000139636,ENSG00000105983   
11        R166.3         R166.3  ENSG00000144233,ENSG00000101935   
12       T01G9.2        T01G9.2                  ENSG00000125149   
13       T04C4.1        T04C4.1  ENSG00000217128